<a href="https://colab.research.google.com/github/SeffySnow/GA-community-detection/blob/main/GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload your text file
uploaded = files.upload()

Saving com-dblp.ungraph.txt to com-dblp.ungraph.txt


In [ ]:
import numpy as np
import random
import pandas as pd

# Constants
POPULATION_SIZE = 30
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1

# Read the graph data from the text file into a DataFrame
graph_data = pd.read_csv("com-dblp.ungraph.txt", delim_whitespace=True)

# Rename columns for better understanding
graph_data.columns = ["FromNodeId", "ToNodeId"]

# Convert the DataFrame into a list of edges
edges = graph_data.values.tolist()

print (graph_data)

# Graph representation
NUM_NODES = max(max(edge) for edge in edges)
adjacency_matrix = np.zeros((NUM_NODES, NUM_NODES))

# Determine the maximum node ID
max_node_id = max(graph_data['FromNodeId'].max(), graph_data['ToNodeId'].max())

# Initialize the adjacency matrix with zeros
adjacency_matrix = np.zeros((max_node_id + 1, max_node_id + 1))

POPULATION_SIZE = 10
CHROMOSOME_LENGTH = 4
NUM_GENERATIONS = 10
MUTATION_RATE = 0.1

def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        individual = np.random.randint(1, NUM_NODES + 1, size=CHROMOSOME_LENGTH)
        population.append(individual)
    return population

def repair_individual(individual):
    repaired_individual = individual.copy()
    for i in range(CHROMOSOME_LENGTH):
        if repaired_individual[i] == i + 1:
            continue
        if adjacency_matrix[i, repaired_individual[i] - 1] != 1:
            neighbors = np.nonzero(adjacency_matrix[i])[0]
            repaired_individual[i] = random.choice(neighbors) + 1
    return repaired_individual

def crossover(parent1, parent2):
    child = np.zeros(CHROMOSOME_LENGTH)
    for i in range(CHROMOSOME_LENGTH):
        if random.random() < 0.5:
            child[i] = parent1[i]
        else:
            child[i] = parent2[i]
    return child

def mutate(individual):
    mutated_individual = individual.copy()
    for i in range(CHROMOSOME_LENGTH):
        if random.random() < MUTATION_RATE:
            neighbors = np.nonzero(adjacency_matrix[i])[0]
            mutated_individual[i] = random.choice(neighbors) + 1
    return mutated_individual

def decode_individual(individual):
    components = []
    visited = np.zeros(NUM_NODES, dtype=bool)
    for node in range(CHROMOSOME_LENGTH):
        if not visited[node]:
            component = []
            stack = [node]
            while stack:
                current_node = stack.pop()
                if not visited[current_node]:
                    visited[current_node] = True
                    component.append(current_node)
                    neighbors = np.nonzero(adjacency_matrix[current_node])[0]
                    stack.extend(neighbors)
            components.append(component)
    return components

def evaluate_individual(individual):
    components = decode_individual(individual)
    score = 0
    for component in components:
        score += calculate_score(adjacency_matrix, component, component, 2)
    return score

def selection(population, fitness_scores):
    if not any(fitness_scores):
        # If all fitness scores are zero, select the first individual
        return [population[0]]
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    return sorted_population[:POPULATION_SIZE // 2]

def create_offspring(selected_parents):
    offspring = []
    while len(offspring) < POPULATION_SIZE - len(selected_parents):
        if len(selected_parents) < 2:
            offspring.extend(initialize_population())  # Add random individuals to reach population size
            break
        parent1, parent2 = random.sample(selected_parents, 2)
        child = crossover(parent1, parent2)
        child = mutate(child)
        offspring.append(child)
    return offspring

# Main Genetic Algorithm loop
population = initialize_population()
print("Initial population size:", len(population))
for generation in range(NUM_GENERATIONS):
    print(f"Generation: {generation + 1}")
    fitness_scores = [evaluate_individual(individual) for individual in population]
    selected_parents = selection(population, fitness_scores)
    print("Selected parents size:", len(selected_parents))
    if not selected_parents:
        print("No parents selected, regenerating population")
        population = initialize_population()
        continue
    offspring = create_offspring(selected_parents)
    print (offspring)
    population = selected_parents + offspring

# Get the best individual after the termination condition is met
best_individual = max(population, key=evaluate_individual)
best_components = decode_individual(best_individual)
print("Best partitioning:")
for i, component in enumerate(best_components, start=1):
    print(f"Community {i}: {component}")


         FromNodeId  ToNodeId
0                 0         2
1                 0      4519
2                 0     23073
3                 0     33043
4                 0     33971
...             ...       ...
1049860      425662    425821
1049861      425669    425696
1049862      425670    425677
1049863      425788    425833
1049864      425875    425876

[1049865 rows x 2 columns]
Initial population size: 10
Generation: 1
Selected parents size: 1
[array([285796,  73850,  37614, 382740]), array([ 93327, 180828, 362325, 127532]), array([354678, 186175, 106201, 109043]), array([288730, 397614, 145718,  98842]), array([172308, 140190,  57242, 389714]), array([278271, 290805, 133049,  34715]), array([319583,  22587, 422696,  30891]), array([  1993, 152710, 416248, 388814]), array([378749, 122614, 363048, 140772]), array([106820, 123082,  99907, 336122])]
Generation: 2
Selected parents size: 1
[array([176824,  10419, 386422, 333382]), array([ 73587, 270398, 227782, 220863]), array([18299